# Ejercicio 2: Overfitting en Python
Este notebook implementa el cálculo de R², R² ajustado y R² fuera de muestra usando modelos polinomiales.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

n = 1000
X = np.random.uniform(0, 1, n)
X.sort()
X = X.reshape(-1, 1)
e = np.random.normal(0, 1, n).reshape(-1, 1)
y = np.exp(4 * X) + e

def mse(y, y_hat):
    return np.mean((y - y_hat) ** 2)

def r2_manual(y, y_hat):
    mse_val = mse(y, y_hat)
    return 1 - mse_val / np.mean(y ** 2)

def r2_adjusted(r2, n, k):
    if k >= n-1:
        return np.nan  # evitar divisiones extrañas
    return 1 - (1 - r2) * (n - 1) / (n - k - 1)

features_list = [1, 2, 5, 10, 20, 50, 100, 200, 500]
r2_full, r2_adj, r2_out, mse_full = [], [], [], []

for p in features_list:
    X_p = np.hstack([X ** i for i in range(1, p + 1)])
    model = LinearRegression()
    model.fit(X_p, y)
    y_hat = model.predict(X_p)

    mse_full.append(mse(y, y_hat))
    r2 = r2_manual(y, y_hat)
    r2_full.append(r2)
    r2_adj.append(r2_adjusted(r2, n, p))

    X_train, X_test, y_train, y_test = train_test_split(X_p, y, test_size=0.25, random_state=123)
    model.fit(X_train, y_train)
    y_pred_test = model.predict(X_test)
    r2_out.append(r2_manual(y_test, y_pred_test))


In [ ]:
fig, ax = plt.subplots(figsize=(7,5))
ax.plot(features_list, r2_full, marker="o")
ax.set_xscale("log")
ax.set_xlabel("Número de features (escala log)")
ax.set_ylabel("R²")
ax.set_title("R² en toda la muestra (manual)")
plt.show()

fig, ax = plt.subplots(figsize=(7,5))
ax.plot(features_list, r2_adj, marker="o", color="orange")
ax.set_xscale("log")
ax.set_xlabel("Número de features (escala log)")
ax.set_ylabel("R² ajustado")
ax.set_title("R² ajustado (manual)")
plt.show()

fig, ax = plt.subplots(figsize=(7,5))
ax.plot(features_list, r2_out, marker="o", color="green")
ax.set_xscale("log")
ax.set_xlabel("Número de features (escala log)")
ax.set_ylabel("R² fuera de muestra")
ax.set_title("R² en test (manual)")
plt.show()

## Evaluación de resultados obtenidos

En los tres gráficos se observa un patrón consistente en cuanto al incremento en el número de variables explicativas eleva rápidamente la capacidad explicativa del modelo hasta valores cercanos a uno, tanto en el R² de la muestra, como en el R² ajustado y el R² fuera de muestra.

**R² en toda la muestra**:  
A medida que se incorporan más regresores (X, X², …, Xᵖ), el modelo logra capturar de manera más precisa la relación no lineal entre X y Y. Este resultado es esperado porque las variables añadidas son relevantes, pues no se trata de ruido, sino de transformaciones con potencias directamente vinculadas con el proceso generador de datos.

**R² ajustado**:  
Normalmente, el R² ajustado penaliza la inclusión de regresores irrelevantes. Sin embargo, en este ejercicio, como todos los regresores contienen información relacionada con Y, la penalización no reduce significativamente el estadístico, que también se mantiene cercano a 1. Esto refleja que la ganancia de ajuste supera con claridad el costo de complejidad.

**R² fuera de muestra**:  
El desempeño en el conjunto de prueba confirma que no hay sobreajuste en sentido estricto. El modelo generaliza muy bien porque los regresores no son espurios; son potencias de X que reproducen la forma funcional subyacente de Y. En un escenario alternativo, donde hubiese regresores aleatorios o no correlacionados con la variable dependiente, el R² externo a la muestra caería cuando aumenta el número de variables, lo cual reflejaría el clásico problema de overfitting.